In [1]:
# # import re
# from collections import defaultdict, Counter
# # from itertools import groupby
# # from nltk import flatten
# # from nltk.util import trigrams as get_trigrams
# from mwu_measures.preprocessing_corpus import clean_bnc_line, preprocess_bnc
# # import mwu_measures
# from mwu_measures.processing_corpus import Corpus
import mwu_measures
from mwu_measures.compute_functions import min_max_norm
from mwu_measures import compute_functions
from mwu_measures import preprocessing_corpus
from mwu_measures import processing_corpus
# from mwu_measures.mwu_functions import get_association, get_entropy_dif
from collections import defaultdict, Counter
# from nltk import FreqDist
import numpy as np
import pandas as pd
import json

In [2]:
# this_corpus = mwu_measures.processing_corpus.process_corpus(corpus_name = 'bnc', corpus_dir='small_corpus.txt', chunk_size=1000000, verbose=True)

In [3]:
this_corpus = processing_corpus.Corpus('bnc')

In [4]:
with open('small_corpus.txt', 'r', encoding="utf-8") as corpus_file:
    i = 0
    raw_lines = corpus_file.readlines(100000)
    ngram_dicts = preprocessing_corpus.preprocess_bnc(raw_lines)

In [ ]:
%timeit this_corpus.add_chunk(ngram_dicts)

In [ ]:
%timeitthis_corpus.add_chunk_polars(ngram_dicts)

In [ ]:
this_corpus.trigram_pl